<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning" style="width: 600px">
</div>

### **Limpieza de datos**

A medida que inspeccionamos y limpiamos nuestros datos, necesitaremos construir varias expresiones de columna y consultas para expresar las transformaciones que aplicaremos a nuestro conjunto de datos.

Las expresiones de columna se construyen a partir de columnas existentes, operadores y funciones integradas. Pueden utilizarse en sentencias **`SELECT`** para expresar transformaciones que crean nuevas columnas.

Muchos comandos de consulta SQL estándar (por ejemplo, **`DISTINCT`**, **`WHERE`**, **`GROUP BY`**, etc.) están disponibles en Spark SQL para expresar transformaciones.

En este notebook, revisaremos algunos conceptos que pueden diferir de otros sistemas a los que estés acostumbrado, así como algunas funciones útiles para operaciones comunes.

Prestaremos especial atención a los comportamientos en torno a los valores **`NULL`**, así como al formateo de strings y campos datetime.

#### **Objetivos de aprendizaje**
Al final de esta lección, deberás ser capaz de:
- Resumir datasets y describir comportamientos null
- Recuperar y eliminar duplicados
- Validar datasets para recuentos esperados, valores perdidos y registros duplicados
- Aplicar transformaciones comunes para limpiar y transformar datos

#### **Run Setup**

El setup script creará los datos y declarará los valores necesarios para que el resto de este notebook se ejecute.

In [ ]:
%run ../Includes/Classroom-Setup-02.4

#### **Resumen de datos**

Trabajaremos con registros de nuevos usuarios de la tabla **`users_dirty`**, que tiene el siguiente esquema:

| field | type | description |
|---|---|---|
| user_id | string | unique identifier |
| user_first_touch_timestamp | long | time at which the user record was created in microseconds since epoch |
| email | string | most recent email address provided by the user to complete an action |
| updated | timestamp | time at which this record was last updated |

Empecemos por contar los valores de cada campo de nuestros datos.

In [ ]:
%sql
SELECT count(*), count(user_id), count(user_first_touch_timestamp), count(email), count(updated)
FROM users_dirty

<center><img src="https://i.postimg.cc/XYs1S6ws/db357.png"></center>

#### **Inspeccionar los datos que faltan**

Basándonos en los recuentos anteriores, parece que hay al menos un puñado de valores nulos en todos nuestros campos.

**NOTA:** Los valores nulos se comportan incorrectamente en algunas funciones matemáticas, incluyendo **`count()`**.

- **`count(col)`** omite valores **`NULL`** al contar columnas o expresiones específicas.
- **`count(*)`** es un caso especial que cuenta el número total de filas (incluyendo filas que son sólo valores **`NULL`**).

Podemos contar los valores nulos de un campo filtrando los registros en los que ese campo es nulo, utilizando:  
**`count_if(col IS NULL)`** o **`count(*)`** con un filtro para donde **`col IS NULL`**. 

Ambas sentencias cuentan correctamente los registros en los que faltan correos electrónicos.

In [ ]:
%sql
SELECT count_if(email IS NULL) FROM users_dirty;
SELECT count(*) FROM users_dirty WHERE email IS NULL;

<center><img src="https://i.postimg.cc/5tj7fKgb/db358.png"></center>

In [ ]:
%python 
from pyspark.sql.functions import col
usersDF = spark.read.table("users_dirty")

usersDF.selectExpr("count_if(email IS NULL)")
usersDF.where(col("email").isNull()).count()

<center><img src="https://i.postimg.cc/DzDC0Qps/db359.png"></center>

#### **Deduplicar filas**
Podemos utilizar **`DISTINCT *`** para eliminar registros realmente duplicados en los que filas enteras contienen los mismos valores.

In [ ]:
%sql
SELECT DISTINCT(*) FROM users_dirty

<center><img src="https://i.postimg.cc/650Ynx1B/db360.png"></center>

In [ ]:
%python
usersDF.distinct().count()

<center><img src="https://i.postimg.cc/fL98931P/db361.png"></center>

#### **Deduplicar filas basándose en columnas específicas**

El siguiente código utiliza **`GROUP BY`** para eliminar registros duplicados basándose en los valores de las columnas **`user_id`** y **`user_first_touch_timestamp`**. (Recordemos que estos dos campos se generan cuando se encuentra por primera vez un usuario determinado, formando así tuplas únicas).

Aquí, estamos utilizando la función agregada **`max`** como un hack to:
- Mantener los valores de las columnas **`email`** y **`updated`** en el resultado de nuestro group by
- Capturar los correos electrónicos no nulos cuando hay varios registros presentes

In [ ]:
%sql
CREATE OR REPLACE TEMP VIEW deduped_users AS 
SELECT user_id, user_first_touch_timestamp, max(email) AS email, max(updated) AS updated
FROM users_dirty
WHERE user_id IS NOT NULL
GROUP BY user_id, user_first_touch_timestamp;

SELECT count(*) FROM deduped_users

<center><img src="https://i.postimg.cc/5yWnq4V2/db362.png"></center>

In [ ]:
%python
from pyspark.sql.functions import max
dedupedDF = (usersDF
    .where(col("user_id").isNotNull())
    .groupBy("user_id", "user_first_touch_timestamp")
    .agg(max("email").alias("email"), 
         max("updated").alias("updated"))
    )

dedupedDF.count()

<center><img src="https://i.postimg.cc/CMbvMqGB/db363.png"></center>

Vamos a confirmar que tenemos el recuento esperado de registros restantes después de la deduplicación basada en valores distintos **`user_id`** y **`user_first_touch_timestamp`**.

In [ ]:
%sql
SELECT COUNT(DISTINCT(user_id, user_first_touch_timestamp))
FROM users_dirty
WHERE user_id IS NOT NULL

<center><img src="https://i.postimg.cc/28m01SGH/db364.png"></center>

In [ ]:
%python
(usersDF
    .dropDuplicates(["user_id", "user_first_touch_timestamp"])
    .filter(col("user_id").isNotNull())
    .count())

<center><img src="https://i.postimg.cc/fy758Gzj/db365.png"></center>

#### **Validar Datasets**
Basándonos en nuestra revisión manual anterior, hemos confirmado visualmente que nuestros recuentos son los esperados.
 
También podemos realizar la validación mediante programación utilizando filtros sencillos y cláusulas **`WHERE`**.

Comprueba que el **`user_id`** de cada fila es único.

In [ ]:
%sql
SELECT max(row_count) <= 1 no_duplicate_ids 
FROM (
      SELECT user_id, count(*) AS row_count
      FROM deduped_users
      GROUP BY user_id
      )

<center><img src="https://i.postimg.cc/qvvQGD9y/db366.png"></center>

In [ ]:
%python
from pyspark.sql.functions import count

display(dedupedDF
    .groupBy("user_id")
    .agg(count("*").alias("row_count"))
    .select((max("row_count") <= 1).alias("no_duplicate_ids")))

<center><img src="https://i.postimg.cc/XY1L1cbh/db367.png"></center>

Confirm that each email is associated with at most one **`user_id`**.

In [ ]:
%sql
SELECT max(user_id_count) <= 1 at_most_one_id FROM (
  SELECT email, count(user_id) AS user_id_count
  FROM deduped_users
  WHERE email IS NOT NULL
  GROUP BY email)

<center><img src="https://i.postimg.cc/Zq9cGBYq/db368.png"></center>

In [ ]:
%python

display(dedupedDF
    .where(col("email").isNotNull())
    .groupby("email")
    .agg(count("user_id").alias("user_id_count"))
    .select((max("user_id_count") <= 1).alias("at_most_one_id")))

<center><img src="https://i.postimg.cc/FK2ZNr1V/db369.png"></center>

#### **Formato de fecha y Regex**
Ahora que hemos eliminado los campos nulos y los duplicados, podemos extraer más valor de los datos.

El código siguiente:
- Escala correctamente y castea **`user_first_touch_timestamp`** en un timestamp válido.
- Extrae la fecha del calendario y la hora de este timestamp en un formato legible.
- Utiliza **`regexp_extract`** para extraer los dominios de la columna email utilizando regex

In [ ]:
%sql
SELECT *, 
  date_format(first_touch, "MMM d, yyyy") AS first_touch_date,
  date_format(first_touch, "HH:mm:ss") AS first_touch_time,
  regexp_extract(email, "(?<=@).+", 0) AS email_domain
FROM (
  SELECT *,
    CAST(user_first_touch_timestamp / 1e6 AS timestamp) AS first_touch 
  FROM deduped_users
)

<center><img src="https://i.postimg.cc/Bb3BPmT8/db370.png"></center>

<center><img src="https://i.postimg.cc/SRG76cdg/db371.png"></center>

In [ ]:
%python
from pyspark.sql.functions import date_format, regexp_extract

display(dedupedDF
    .withColumn("first_touch", (col("user_first_touch_timestamp") / 1e6).cast("timestamp"))
    .withColumn("first_touch_date", date_format("first_touch", "MMM d, yyyy"))
    .withColumn("first_touch_time", date_format("first_touch", "HH:mm:ss"))
    .withColumn("email_domain", regexp_extract("email", "(?<=@).+", 0))
)

<center><img src="https://i.postimg.cc/Y2N3Bw7Q/db372.png"></center>

In [ ]:
%python
from pyspark.sql.functions import date_format, regexp_extract

display(dedupedDF
    .withColumn("first_touch", (col("user_first_touch_timestamp") / 1e6).cast("timestamp"))
    .withColumn("first_touch_date", date_format("first_touch", "MMM d, yyyy"))
    .withColumn("first_touch_time", date_format("first_touch", "HH:mm:ss"))
    .withColumn("email_domain", regexp_extract("email", "(?<=@).+", 0))
)

<center><img src="https://i.postimg.cc/VLYFYHsp/db373.png"></center>

Ejecute la siguiente celda para eliminar las tablas y archivos asociados a esta lección.

In [ ]:
%python
DA.cleanup()

-sandbox
&copy; 2022 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the <a href="https://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/>
<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a> | <a href="https://help.databricks.com/">Support</a>